<a href="https://colab.research.google.com/github/renanreyc/IA-UEPB/blob/main/Aula_Estudo_de_Caso_Titanic_2022_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Problema de predição de sobreviventes do Titanic

In [2]:
# instale e importe o pandas-profiling
from pandas_profiling import ProfileReport

# kaggle.com/c/titanic/data

Quando o Titanic afundou, 1502 dos 2224 passageiros e tripulantes morreram. Uma das principais razões para esta grande quantidade de baixas foi a falta de botes salva-vidas neste autoproclamado navio "inafundável".

Aqueles que assistiram ao filme sabem que alguns indivíduos eram mais propensos a sobreviver ao naufrágio (Rose sortuda) do que outros (o pobre Jack). Neste curso, você aprenderá como aplicar técnicas de aprendizado de máquina para prever as chances de um passageiro sobreviver usando o Python.

Vamos começar carregando o conjunto de treinamento e testes em seu ambiente Python. Você usará o conjunto de treinamento para criar seu modelo e o conjunto de testes para validá-lo. Os dados são armazenados na web como `arquivos csv`; suas URLs já estão disponíveis como sequências de caracteres no código de amostra. Você pode carregar estes dados com o método `read_csv()` da biblioteca Pandas.

In [4]:
# Importando a biblioteca Pandas
import pandas as pd

In [5]:
# Carregando os conjuntos de treino e teste para criar dois DataFrames
train_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/train.csv"
train = pd.read_csv(train_url)

test_url = "http://s3.amazonaws.com/assets.datacamp.com/course/Kaggle/test.csv"
test = pd.read_csv(test_url)

# Imprimindo as primeiras linhas dos conjuntos de teste e treino do dataframe
print(train.head(3))
print("\n#############################\n")
print(test.head(3))

   PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  

#############################

   PassengerId  Pclass                              Name     Sex   Age  SibSp  \
0          892       3                  Kelly, Mr. James    male  34.5      0   
1          893       3  Wilkes, Mrs. James (Ellen Needs)  female  47.0      1   
2          894       2         Myles, Mr. Thomas Francis   

## Entendendo seus dados
Antes de começar com a análise, é importante entender a estrutura dos seus dados. Ambos `test` e `train` são objetos DataFrame, o modo como Pandas representa os conjuntos de dados. Você pode explorar facilmente um DataFrame usando o método `.describe()` . `.describe()` resume as colunas/características do DataFrame, incluindo a contagem de observações, média, máx e assim por diante. Outra dica útil é observar as dimensões do DataFrame. Isto é feito solicitando o atributo `.shape` do seu objeto DataFrame. (ex. `seus_dados.shape`)

O conjunto de treinamento e de teste já está disponível no espaço de trabalho (workplace), como `train` e` test`. Aplique o método `.describe ()` e imprima o atributo `.shape` do conjunto de treinamento. 

### <font color = "000080">Qual das seguintes afirmações está correta?</font>
1. O conjunto de treinamento tem 891 observações e 12 variáveis, a contagem total para idade é de 714. ✅
2. O conjunto de treinamento tem 891 observações e 11 variáveis, a contagem total para idade é de 891.
3. O conjunto de treinamento tem 418 observações e 12 variáveis, a contagem total para idade é de 714.
4. O conjunto de treinamento tem 418 observações e 11 variáveis, a contagem total para idade é de 891.

In [53]:
train.shape
train.describe()



,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,Child
count,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,891.000000,714.000000
mean,446.000000,0.383838,2.308642,29.361582,0.523008,0.381594,32.204208,0.158263
std,257.353842,0.486592,0.836071,13.019697,1.102743,0.806057,49.693429,0.365244
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,22.000000,0.000000,0.000000,7.910400,0.000000
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200,0.000000
75%,668.500000,1.000000,3.000000,35.000000,1.000000,0.000000,31.000000,0.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200,1.000000


In [7]:
test.shape
test.describe()

,PassengerId,Pclass,Age,SibSp,Parch,Fare
count,418.000000,418.000000,332.000000,418.000000,418.000000,417.000000
mean,1100.500000,2.265550,30.272590,0.447368,0.392344,35.627188
std,120.810458,0.841838,14.181209,0.896760,0.981429,55.907576
min,892.000000,1.000000,0.170000,0.000000,0.000000,0.000000
25%,996.250000,1.000000,21.000000,0.000000,0.000000,7.895800
50%,1100.500000,3.000000,27.000000,0.000000,0.000000,14.454200
75%,1204.750000,3.000000,39.000000,1.000000,0.000000,31.500000
max,1309.000000,3.000000,76.000000,8.000000,9.000000,512.329200


In [22]:
# utilize a chamada de ProfileReport (do pandas-profiling)
# para gerar o relatório sobre o conjunto de dados
profile = ProfileReport(train, title="Pandas Profiling Report")

# uma célula para train e outra célula para test

TypeError: ignored

## Rose vs Jack, ou Feminino vs Masculino
Quantas pessoas em seu conjunto de dados de treinamento sobreviveram ao desastre do Titanic? Para ver isso, você pode usar o método `value_counts ()` em combinação com a notação de colchetes padrão para selecionar uma única coluna de um DataFrame:

```# números absolutos
train["Survived"].value_counts()```

```# porcentagens
train["Survived"].value_counts(normalize = True)```

Se você executar esses comandos no console, observará que 549 pessoas morreram (62%) e 342 sobreviveram (38%). Uma maneira simples de prever heuristicamente poderia ser: "maioria ganha". Isto significa que você vai prever cada observação invisível para não sobreviver.

Para mergulhar um pouco mais fundo, podemos realizar contagens e cálculos percentuais semelhantes em subconjuntos da coluna Survived. Por exemplo, talvez gênero também pudesse desempenhar um papel? Você pode explorar isso usando o método `.value_counts ()` para uma comparação bidirecional sobre o número de homens e mulheres que sobreviveram, com esta sintaxe:

``` train["Survived"][train["Sex"] == 'male'].value_counts()
train["Survived"][train["Sex"] == 'female'].value_counts() ```

Para obter proporções, você pode passar novamente no argumento normalize = True para o método `.value_counts ()`.

In [8]:
# Calcule e imprima as taxas de sobrevivência em números absolutos usando o método values_counts ().
# Passageiros que sobreviveram vs passageiros que faleceram (0 = mortes e 1 = sobreviventes)
print(train["Survived"].value_counts())

0    549
1    342
Name: Survived, dtype: int64


In [9]:
# Calcule e imprima as taxas de sobrevivência como proporções configurando o argumento normalize como True.
print(train["Survived"].value_counts(normalize = True))

0    0.616162
1    0.383838
Name: Survived, dtype: float64


In [10]:
# Repita os mesmos cálculos, mas em subconjuntos de sobrevivências baseadas em sexo.
# Homens que sobreviveram vs homens que faleceram
print(train["Survived"][train["Sex"] == 'male'].value_counts())

0    468
1    109
Name: Survived, dtype: int64


In [11]:
# Mulheres que sobreviveram vs mulheres que faleceram
print(train["Survived"][train["Sex"] == 'female'].value_counts())

1    233
0     81
Name: Survived, dtype: int64


In [13]:
# Taxa de sobrevivência masculina normalizada
print(train["Survived"][train["Sex"] == 'male'].value_counts(normalize = True))

0    0.811092
1    0.188908
Name: Survived, dtype: float64


In [14]:
# Taxa de sobrevivência feminina normalizada
print(train["Survived"][train["Sex"] == 'female'].value_counts(normalize = True))

1    0.742038
0    0.257962
Name: Survived, dtype: float64


## A idade influencia em algo?
Outra variável que pode influenciar a sobrevivência é a idade; uma vez que é provável que as crianças foram salvas primeiro. Você pode testar isso criando uma nova coluna com uma variável categórica `Child`. 'Child' terá o valor 1 nos casos em que a idade for menor que 18 e o valor 0 nos casos em que a idade for maior ou igual a 18.

Para adicionar essa nova variável, você precisa fazer duas coisas (i) criar uma nova coluna e (ii) fornecer os valores para cada observação (ou seja, linha) com base na idade do passageiro.

Adicionar uma nova coluna com Pandas em Python é fácil e pode ser feito através da seguinte sintaxe:

```your_data["new_var"] = 0```

Este código criaria uma nova coluna no DataFrame `train` chamado `new_var` com `0` para cada observação.

Para definir os valores com base na idade do passageiro, você faz um teste booleano dentro do operador de colchetes. Com o operador `[]`, você cria um subconjunto de linhas e atribui um valor a uma determinada variável desse subconjunto de observações. Por exemplo,

``` train["new_var"][train["Fare"] > 10] = 1 ```

daria um valor de `1` à variável `new_var` para o subconjunto de passageiros com tarifas superiores a `10`. Lembre-se que `new_var` tem um valor de `0` para todos os outros valores (incluindo valores omissos).

Uma nova coluna chamada `Child` no dataframe ` train` foi criada para você e recebe o valor `NaN` para todas as observações.

In [20]:
# Defina os valores de Child para 1 se a idade do passageiro for menor que 18 anos.
# Crie a coluna Child e a atribua 'NaN'
train["Child"] = float('NaN')
train.head(10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Child
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S,NaN
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C,NaN
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,NaN
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S,NaN
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S,NaN
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q,NaN
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S,NaN
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S,NaN
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S,NaN
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C,NaN


In [21]:
# Atribua 1 aos passageiros menores que 18 e 0 àqueles maiores (ou com 18).
train["Child"][train["Age"] < 18] = 1
train["Child"][train["Age"] >= 18] = 0

# Mostre na tela a coluna nova: Child.
print(train["Child"])

0      0.0
1      0.0
2      0.0
3      0.0
4      0.0
      ... 
886    0.0
887    0.0
888    NaN
889    0.0
890    0.0
Name: Child, Length: 891, dtype: float64


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [23]:
# Imprimir taxas de sobrevivência normalizadas
#  para passageiros menores que 18
print(train["Survived"][train["Child"] == 1].value_counts(normalize = True))

1    0.539823
0    0.460177
Name: Survived, dtype: float64


In [24]:
# Imprimir taxas de sobrevivência normalizadas
#  para passageiros maiores que 18
print(train["Survived"][train["Child"] == 0].value_counts(normalize = True))

0    0.618968
1    0.381032
Name: Survived, dtype: float64


## Primeira Predição

Em um dos exercícios anteriores, você descobriu que, no seu conjunto de treinamento, o sexo feminino tinha mais de 50% de chance de sobreviver e o sexo masculino tinha menos de 50% de chance de sobreviver. Portanto, você pode usar essa informação para a sua primeira previsão: todas as pessoas do sexo feminino no conjunto de testes sobrevivem e todos do sexo masculino no mesmo conjunto morrem.

Você utiliza seu conjunto de testes para validar suas previsões. Você pode ter visto que, ao contrário do conjunto de treinamento, o conjunto de testes não possui uma coluna `Survived`. Você adiciona essa coluna usando seus valores previstos. Em seguida, ao enviar seus resultados, o Kaggle usará **sua** variável (= suas previsões) para avaliar seu desempenho.

In [25]:
# Criar uma cópia do conjunto de testes: test_one
test_one = test.copy()

In [26]:
# Adicione uma coluna adicional, Survived, e inicialize com zero.
# Inicialize uma coluna Survived com 0
test_one["Survived"] = 0

In [28]:
# Defina Survived com 1 se Sex for igual a "female" e imprima a coluna `Survived` de` test_one`
test_one["Survived"][test_one["Sex"] == 'female'] = 1
test_one["Survived"][test_one["Sex"] == 'male'] = 0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [29]:
print(test_one)

     PassengerId  Pclass                                          Name  \
0            892       3                              Kelly, Mr. James   
1            893       3              Wilkes, Mrs. James (Ellen Needs)   
2            894       2                     Myles, Mr. Thomas Francis   
3            895       3                              Wirz, Mr. Albert   
4            896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)   
..           ...     ...                                           ...   
413         1305       3                            Spector, Mr. Woolf   
414         1306       1                  Oliva y Ocana, Dona. Fermina   
415         1307       3                  Saether, Mr. Simon Sivertsen   
416         1308       3                           Ware, Mr. Frederick   
417         1309       3                      Peter, Master. Michael J   

        Sex   Age  SibSp  Parch              Ticket      Fare Cabin Embarked  \
0      male  34.5      0      0

In [30]:
# Imprima a coluna Survived de previsões do conjunto de dados test_one.
print(test_one["Survived"])

0      0
1      1
2      0
3      0
4      1
      ..
413    0
414    1
415    0
416    0
417    0
Name: Survived, Length: 418, dtype: int64


In [32]:
test_one["PassengerId"].values

array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
        903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
        914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
        925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
        936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
        947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
        958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
        969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
        980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
        991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
       1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
       1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
       1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
       1035, 1036, 1037, 1038, 1039, 1040, 1041, 10

In [33]:
test_one["Survived"].values

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [34]:
submit = test_one["PassengerId"].values, test_one["Survived"].values
submit

(array([ 892,  893,  894,  895,  896,  897,  898,  899,  900,  901,  902,
         903,  904,  905,  906,  907,  908,  909,  910,  911,  912,  913,
         914,  915,  916,  917,  918,  919,  920,  921,  922,  923,  924,
         925,  926,  927,  928,  929,  930,  931,  932,  933,  934,  935,
         936,  937,  938,  939,  940,  941,  942,  943,  944,  945,  946,
         947,  948,  949,  950,  951,  952,  953,  954,  955,  956,  957,
         958,  959,  960,  961,  962,  963,  964,  965,  966,  967,  968,
         969,  970,  971,  972,  973,  974,  975,  976,  977,  978,  979,
         980,  981,  982,  983,  984,  985,  986,  987,  988,  989,  990,
         991,  992,  993,  994,  995,  996,  997,  998,  999, 1000, 1001,
        1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 1010, 1011, 1012,
        1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022, 1023,
        1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1032, 1033, 1034,
        1035, 1036, 1037, 1038, 1039, 

## Introdução as árvores de decisão

No capítulo anterior, você fez toda uma segmentação para encontrar subconjuntos que têm uma chance maior de sobreviver. Uma árvore de decisão automatiza esse processo para você e gera um modelo de classificação ou classificador.

Conceitualmente, o algoritmo da árvore de decisão começa com todos os dados no nó raiz e varre todas as variáveis para a melhor divisão. Quando uma variável é escolhida, você faz a divisão e desce um nível (ou um nó) e repete. Os nós finais na parte inferior da árvore de decisão são conhecidos como nós terminais, e o voto majoritário das observações nesse nó determina como prever novas observações que param nesse nó terminal.

Primeiro, vamos importar as bibliotecas necessárias:

    Import the numpy library as np
    From sklearn import the tree


In [35]:
# Importe a biblioteca Numpy
import numpy as np
# Importe 'tree' da biblioteca scikit-learn
from sklearn import tree

## Limpando e formatando seus dados

Antes de começar a construir suas árvores, você precisa sujar as mãos e limpar os dados para poder usar todos os recursos disponíveis para você. No primeiro capítulo, vimos que a variável `Age` tinha algum valor faltante (missing value). A falta de dados por si só já é um assunto para investigar mais profundamente, mas nós usaremos uma técnica de imputação simples onde substituímos cada valor perdido com a `mediana` dos valores presentes.

    train["Age"] = train["Age"].fillna(train["Age"].median())

Outro problema é que as variáveis `Sex` e` Embarked` são categóricas **mas** em formato não numérico. Assim, precisaremos atribuir a cada classe um inteiro único para que o Python possa manipular as informações. `Embarked` também tem alguns valores perdidos que você deve imputar com a classe mais comum de embarque, que é `" S "`.

In [36]:
train["Age"] = train["Age"].fillna(train["Age"].median())
test_one["Age"] = test_one["Age"].fillna(test_one["Age"].median())

In [ ]:
# Descomentem e testem quando finalizarem essa aula
# Testem após a linha anterior para ver o que acontece
# train["Age"] = train["Age"].dropna()

# Resposta: remove os valores ausentes da coluna "Age"

In [37]:
# Converta os grupos mulheres e homens para a forma inteira
# Atribua o valor inteiro 1 para todas as mulheres
train["Sex"][train["Sex"] == "male"] = 0
train["Sex"][train["Sex"] == "female"] = 1
test_one["Sex"][test_one["Sex"] == "male"] = 0
test_one["Sex"][test_one["Sex"] == "female"] = 1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [39]:
# "Imputar" a variável Embarked
# "Imputar" os missing values em Embarked com classe S. Use o método .fillna().
train["Embarked"] = train["Embarked"].fillna("S")
test_one["Embarked"] = test_one["Embarked"].fillna("S")

In [40]:
# Substitua cada classe de Embarked com inteiros únicos.
# 0 para S, 1 para C, e 2 para Q.
# Converta as classes de Embarked para inteiros

train["Embarked"][train["Embarked"] == "S"] = 0
train["Embarked"][train["Embarked"] == "C"] = 1
train["Embarked"][train["Embarked"] == "Q"] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [41]:
# Substitua cada classe de Embarked com inteiros únicos.
# 0 para S, 1 para C, e 2 para Q.
# Converta as classes de Embarked para inteiros

test_one["Embarked"][test_one["Embarked"] == "S"] = 0
test_one["Embarked"][test_one["Embarked"] == "C"] = 1
test_one["Embarked"][test_one["Embarked"] == "Q"] = 2

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [42]:
# Mostre as colunas Sex e Embarked
print(train["Sex"])
print(train["Embarked"])

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    1
889    0
890    0
Name: Sex, Length: 891, dtype: object
0      0
1      1
2      0
3      0
4      0
      ..
886    0
887    0
888    0
889    1
890    2
Name: Embarked, Length: 891, dtype: object


## Criando sua primeira árvore de decisão

Utilizando as bibliotecas `scikit-learn` e `numpy` construiremos nosso primeiro modelo de árvore de decisão. `scikit-learn` pode ser utilizada para criar objetos tipo árvore a partir da classe `DecisionTreeClassifier`. Os métodos que utilizaremos recebem arrays `numpy` como entradas e então precisamos acessar nossas entradas com o `DataFrame` que já temos. Construímos a árvore de decisão da seguinte forma:

* target: Um array unidimensional `numpy` contendo o alvo/resposta do conjunto de treinamento. (Survival neste caso)
* features: Um array multidimensional `numpy` contendo as features/preditores do conjunto de treinamento. (ex. Sex, Age)

Veja o trecho de código abaixo para entender como deve parecer essa etapa:

    target = train["Survived"].values

    features = train[["Sex", "Age"]].values

    my_tree = tree.DecisionTreeClassifier()

    my_tree = my_tree.fit(features, target)


Uma forma rápida de ver o resultado da árvore de decisão é ver qual a importância das features incluídas no treinamento. Isto é feito requisitando o atributo `.feature_importances_ attribute` do seu objeto (da árvore). Outra métrica rápida é medir a acurácia que você pode computar utilizando a função `.score()` com `features_one` e `target` como argumentos.

Certo?! É hora de você construir sua primeira árvore de decisão em Python! Os dados de treinamento `train` e de teste `testing` já estão disponíveis.

In [43]:
# Mostrar as informações sobre os dados de treinamento
print(train)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris   0  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...   1  38.0      1   
2                               Heikkinen, Miss. Laina   1  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)   1  35.0      1   
4                             Allen, Mr. William Henry   0  35.0      0   
..                                                 ...  ..   ...    ...   
886               

In [44]:
# Construa os arrays numpy de alvo/objetivo `target` e `features_one`. 
# O objetivo é baseado na coluna Survived em train. 
# O vetor com a entrada baseia-se nas variáveis Passenger, Class, Sex, Age, e Passenger Fare
# Crie os arrays numpy: target, features_one
target = train["Survived"].values  # Complete com o nome correto da coluna
features_one = train[["Pclass", "Sex", "Age", "Fare", "SibSp", "Embarked"]].values
print(features_one)

[[3 0 22.0 7.25 1 0]
 [1 1 38.0 71.2833 1 1]
 [3 1 26.0 7.925 0 0]
 ...
 [3 1 28.0 23.45 1 0]
 [1 0 26.0 30.0 0 1]
 [3 0 32.0 7.75 0 2]]


In [45]:
# Construa a árvore de decisão my_tree_one para predizer a sobrevivência utilizando:
#  features_one e target
my_tree_one = tree.DecisionTreeClassifier()
my_tree_one = my_tree_one.fit(features_one, target)


In [46]:
# Veja a importância das variáveis/features na sua árvore e calcule a pontuação
# Veja a importância e pontuação das features
print(my_tree_one.feature_importances_)
# print(my_tree_one.score(features_one, target))  # Descomente aqui e teste

[0.11221908 0.31194178 0.23493517 0.26572    0.05226001 0.02292396]


## Interpretando sua árvore de decisão

O atributo `feature_importances_` torna simples interpretar o que os preditores representam.
Com base em sua árvore de decisão, qual variável é mais importante em determinar se um passageiro sobreviveu ou não? Seu modelo (`my_tree_one`) é disponível no console.

### <font color = "000080">Resposta: "______"</font>

## Predição e submissão no Kaggle

Para submeter no Kaggle você precisa predizer as taxas de sobrevivência com base no conjunto de teste.  Com nossa árvore de decisão, nós podemos fazer uso de algumas funções simples para "gerar" nossa resposta.

Primeiro, utilize o método `.predict()`. Você adiciona o modelo (`my_tree_one`), os valores das features do conjunto de dados para as predições que precisam ser feitas (`test`). Para extrair as features precisamos criar um array `numpy` da mesma maneira como fizemos no treinamento do modelo. No entanto, necessitamos ter cuidado com uma pequena mas importante questão. Falta algum valor em `Fare` que precisa ser "imputado".

Depois, você precisa ter certeza que sua saída está de acordo com os requisitos de submissão do Kaggle: um arquivo csv com exatamente 418 entradas e duas colunas: `PassengerId` e `Survived`. Então utilize o código advindo para fazer um novo `DataFrame` utilizando `DataFrame()`, e crie um arquivo csv usando o método `to_csv()` do Pandas.

In [48]:
# Impute o missing value de Fare na linha 153 com a mediana da coluna.
test_one.Fare[152] = test_one["Fare"].median()
print(test_one.Fare[152])

14.4542


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [49]:
# Extraia os atributos/features do conjunto de teste: Pclass, Sex, Age, Fare, SibSp e Embarked.
test_features = test_one[["Pclass", "Sex", "Age", "Fare", "SibSp", "Embarked"]].values

In [50]:
# Faça sua predição utilizando o conjunto de teste: test_features
my_prediction = my_tree_one.predict(test_features)
my_prediction

array([0, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 1,
       0, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1,
       1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1,
       1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [51]:
# Crie um data frame com duas colunas: PassengerId e Survived.
# Survived contém suas predições
PassengerId = np.array(test["PassengerId"]).astype(int)
my_solution = pd.DataFrame(my_prediction, PassengerId, columns = ["Survived"])
print(my_solution)

      Survived
892          0
893          0
894          1
895          1
896          1
...        ...
1305         0
1306         1
1307         0
1308         0
1309         1

[418 rows x 1 columns]


In [54]:
# Verifique se o seu data frame possui 418 entradas
print(my_solution.shape)

(418, 1)


In [55]:
# Salve sua solução em um arquivo csv com o nome my_solution_one.csv
my_solution.to_csv("my_solution_one.csv", index_label = ["PassengerId"])
print(my_solution)

      Survived
892          0
893          0
894          1
895          1
896          1
...        ...
1305         0
1306         1
1307         0
1308         0
1309         1

[418 rows x 1 columns]


## Overfitting e como controlar isso

Quando você criou sua primeira árvore de decisão, os argumentos padrão para `max_depth` e `min_samples_split` foram configurados como `None`. Isso significa que nenhum limite para a profundidade de sua árvore foi definido. Isso é uma coisa boa, certo? Não tão rápido. Provavelmente temos um overfitting. Isso significa que, embora seu modelo descreva os dados de treinamento muito bem, ele não generaliza para novos dados, o que é francamente o ponto de predição. Basta olhar para os resultados de envio do Kaggle para o modelo simples baseado em gênero (Gender) e a árvore de decisão complexa. Qual é o melhor?

Talvez possamos melhorar o modelo de sobreajuste criando um modelo menos complexo? Em `DecisionTreeRegressor`, a profundidade do nosso modelo é definida por dois parâmetros: - o parâmetro `max_depth` determina quando a divisão da árvore de decisão finaliza. - o parâmetro `min_samples_split` monitora a quantidade de observações em um intervalo. Se um certo limite não for atingido (por exemplo, mínimo de 10 passageiros), nenhuma divisão adicional pode ser feita.

Ao limitar a complexidade de sua árvore de decisão, você aumentará sua generalidade e, portanto, sua utilidade para previsões!

In [56]:
# Incluir: irmãos/cônjuges a bordo, pais/filhos a bordo e atributos Embarked em um novo conjunto de atributos.
# Criar uma nova matriz com os recursos adicionados: features_two
features_two = train[["Pclass","Age","Sex","Fare", "SibSp", "Parch", "Embarked"]].values

In [57]:
# Ajuste sua segunda árvore my_tree_two com os novos atributos e controle a complexidade do modelo alternando ...
# ...os argumentos max_depth e min_samples_split.
# Controle o sobreajuste (overfitting) definindo "max_depth" para 10 e "min_samples_split" para 5: my_tree_two

max_depth = 10
min_samples_split = 5
my_tree_two = tree.DecisionTreeClassifier(max_depth = max_depth,
                                          min_samples_split = min_samples_split,
                                          random_state = 1)
my_tree_two = my_tree_two.fit(features_two, target)

In [58]:
# Mostrar o escore da nova árvore de decisão
print(my_tree_two.score(features_two, target))

0.9057239057239057


In [ ]:
# Submeter a segunda solução!

## Feature-engineering para nosso conjunto de dados do Titanic

Data Science é uma arte que se beneficia de um elemento humano. Vamos trabalhar com feature engineering: desenvolva criativamente seus próprios atributos combinando as diferentes variáveis existentes.

Embora feature engineering seja uma disciplina em si, muito ampla para ser abordada aqui em detalhes, você verá um exemplo simples criando seu próprio novo atributo preditivo: `family_size`.

Uma suposição válida é que famílias maiores precisam de mais tempo para se reunir em um navio que está naufragando e, portanto, têm menor probabilidade de sobreviver. O tamanho da família é determinado pelas variáveis `SibSp` e `Parch`, que indicam o número de membros da família com quem um determinado passageiro está viajando. Então, ao fazer featuring engineering, você adiciona uma nova variável `family_size`, que é a soma de `SibSp` e `Parch` mais um (a própria observação), ao conjunto `test` e `train`.



In [59]:
# Crie um novo conjunto de treinamento train_two que difere de train apenas por ter uma coluna extra ...
# ... com sua variável family_size.
# Crie train_two com a recém-definida family_size

train_two = train.copy()
train_two["family_size"] = train["SibSp"] + train["Parch"] + 1

In [61]:
# Adicione family_size além de ...
# ... Pclass, Sex, Age, Fare, SibSp e Parch para features_three.
# Crie um novo conjunto de atributos com esses elementos

features_three = train_two[["Pclass", "Sex", "Age", "Fare", "SibSp", "Parch", "family_size"]].values

In [62]:
# Crie uma nova árvore de decisão como my_tree_three e ajuste a árvore de decisão com seu novo conjunto features_three.
# Em seguida, verifique a pontuação da árvore de decisão.
# Defina o classificador de árvore e ajuste o modelo

my_tree_three = tree.DecisionTreeClassifier()
my_tree_three = my_tree_three.fit(features_three, target)

In [63]:
# Mostrar o escore dessa árvore de decisão
print(my_tree_three.score(features_three, target))

0.9797979797979798


In [ ]:
# Submeter a terceira solução!

## Uma análise com random forest em Python

Um estudo detalhado de Random Forests levaria este estudo de caso um pouco longe demais. No entanto, como é uma técnica de aprendizado de máquina usada com frequência, obter um entendimento geral do Python não fará mal.

Em termos gerais, a técnica Random Forest lida com o problema de overfitting que você enfrentou com as árvores de decisão. Ela desenvolve várias árvores de classificação (muito profundas) usando o conjunto de treinamento. No momento da predição, cada árvore é usada para fazer uma previsão e cada resultado é contado como um voto. Por exemplo, se você treinou 3 árvores com 2 dizendo que um passageiro no conjunto de teste sobreviverá e 1 dizendo que não, o passageiro será classificado como um sobrevivente. Essa abordagem de overtraining de árvores, mas tendo o voto da maioria contado como a decisão de classificação real, evita overfitting.

Construir uma floresta aleatória em Python é quase o mesmo que construir uma árvore de decisão. Existem duas diferenças principais, no entanto. Em primeiro lugar, uma classe diferente é usada. E em segundo lugar, um novo argumento é necessário. Além disso, precisamos importar a biblioteca necessária do `scikit-learn`.

* Use a classe `RandomForestClassifier()` em vez da classe `DecisionTreeClassifier()`.
* `n_estimators` precisa ser definido ao usar a classe `RandomForestClassifier()`. Este argumento permite que você defina o número de árvores que deseja plantar e calcular a média.

Os dados de treinamento e teste mais recentes estão pré-carregados para você.

In [64]:
# Importe `RandomForestClassifier`
from sklearn.ensemble import RandomForestClassifier

In [65]:
# Queremos as variáveis Pclass, Age, Sex, Fare,SibSp, Parch e Embarked
features_forest = train[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values

In [66]:
# Construir o preditor com n_estimators igual a 100.
# Construindo e ajustando my_forest
forest = RandomForestClassifier(max_depth = 10,
                                min_samples_split=2,
                                n_estimators = 100,
                                random_state = 1)
# Ajuste seu modelo de floresta aleatória com entradas features_forest e target.
my_forest = forest.fit(features_forest, target)

In [67]:
# Mostre a pontuação ajustada
print(my_forest.score(features_forest, target))

0.9393939393939394


In [68]:
test_one.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Survived
0,892,3,"Kelly, Mr. James",0,34.5,0,0,330911,7.8292,NaN,2,0
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",1,47.0,1,0,363272,7.0000,NaN,0,1
2,894,2,"Myles, Mr. Thomas Francis",0,62.0,0,0,240276,9.6875,NaN,2,0
3,895,3,"Wirz, Mr. Albert",0,27.0,0,0,315154,8.6625,NaN,0,0
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",1,22.0,1,1,3101298,12.2875,NaN,0,1


In [73]:
# Calcule as previsões do classificador nos atributos do conjunto de teste selecionado.
# Calcule as previsões em nossos recursos de conjunto de teste e imprima o comprimento do vetor de previsão

test_features = test_one[["Pclass", "Age", "Sex", "Fare", "SibSp", "Parch", "Embarked"]].values
pred_forest = my_forest.predict(test_features)
print(len(pred_forest))

418


## Interpretando e comparando

Lembra como olhamos para o atributo `.feature_importances_` para as árvores de decisão? Bem, você também pode solicitar o mesmo atributo de sua floresta aleatória e interpretar a relevância das variáveis incluídas. Você também pode querer comparar os modelos de uma maneira rápida e fácil. Para isso, podemos usar o método `.score ()`. O método `.score ()` pega os atributos `data` e o vetor `target` e calcula a precisão de `mean` do seu modelo. Você pode aplicar este método às árvores **forest** e **individual**. Lembre-se de que essa medida deve ser alta, mas não extrema, porque isso seria um sinal de **overfitting**.

Para este exercício, você tem `my_forest` e `my_tree_two` disponíveis para você. Os arrays `features` e `target` também estão prontos para uso.

In [74]:
# Explore o atributo feature_importances_ e mostre na tela
print(my_tree_two.feature_importances_)
print(my_tree_three.feature_importances_)
print(my_forest.feature_importances_)

[0.14130255 0.17906027 0.41616727 0.17938711 0.05039699 0.01923751
 0.0144483 ]
[0.11264598 0.31088095 0.23178944 0.2391215  0.04020843 0.01821674
 0.04713697]
[0.10384741 0.20139027 0.31989322 0.24602858 0.05272693 0.04159232
 0.03452128]


In [75]:
# Compare a pontuação média de precisão dos dois modelos
# Calcular e imprimir a pontuação média de precisão para ambos os modelos
print(my_tree_two.score(features_two, target))
print(my_tree_three.score(features_two, target))
print(my_forest.score(features_two, target))

0.9057239057239057
0.5398428731762065
0.9393939393939394


## Concluir e submeter

Com base em sua descoberta no exercício anterior, determine qual recurso foi mais importante e para qual modelo. Após este exercício final, você poderá enviar seu modelo de floresta aleatória para o Kaggle! 
Use `my_forest`, `my_tree_two` e `feature_importances_` para responder a pergunta.

### <font color = "000080">Resposta: O atributo mais importante foi **Sex**, mas foi mais significativo para o modelo **my_tree_two**.</font>


## Referência

https://www.datacamp.com/community/open-courses/kaggle-python-tutorial-on-machine-learning